# Analysis of Daily Precipitation Data in Tenerife
This notebook analyzes daily precipitation data from the ERA5 reanalysis dataset for Tenerife.
### Objectives:
- Load and preprocess precipitation data.
- Compute statistical summaries and visualize distributions.
- Fit a **Gamma distribution** to precipitation data.
- Perform **Extreme Value Analysis (EVA)** to model extreme precipitation events.

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import gamma
from pyextremes import EVA
import pandas as pd
import gdown

## Loading Precipitation Data
We use **ERA5 reanalysis daily precipitation data** for Tenerife from **1950 - 2000**.

In [ ]:
#'GOOGLE_DRIVE_LINK' to NetCDF file
url = 'https://drive.google.com/file/d/1ljDfwAeyfLNMsArQDOFP9-7y4EC7Brxq/view?usp=drive_link'
# Download the NetCDF file
file_path = 'data/input/ERA5_Tenerife_total_precipitation_day_1950_2000.nc'
gdown.download(url, file_path, quiet=False, fuzzy=True)

In [ ]:
file_path = 'data/input/ERA5_Tenerife_total_precipitation_day_1950_2000.nc'
daily_precipitation = xr.open_dataset(file_path)
daily_precipitation.attrs['units'] = 'mm/d'
years = daily_precipitation['time'].dt.year.values
daily_precipitation

## Selecting a Specific Grid Cell
We extract data for a specific location based on latitude and longitude.

In [ ]:
latitude_target = 28.25
longitude_target = -16.76
daily_precipitation = daily_precipitation.sel(latitude=latitude_target, longitude=longitude_target, method='nearest')

## Monthly Precipitation Analysis
We calculate the monthly sums and compute the multi-annual mean precipitation per month.

In [ ]:
monthly_sum_precipitation = daily_precipitation.resample(time='1M').sum(dim='time')
annual_mean_precipitation_monthly_sum = monthly_sum_precipitation.groupby('time.month').mean(dim='time')
df = annual_mean_precipitation_monthly_sum.to_dataframe()
df['tp'].plot(kind='bar', figsize=(10, 6))
plt.title('Mean Monthly Precipitation')
plt.xlabel('Month')
plt.ylabel('Total Precipitation [mm]')
plt.xticks(range(12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.grid(axis='y')
plt.show()

## Violin Plot of Daily Precipitation by Month
A **violin plot** allows us to analyze the distribution of precipitation values within each month.

In [ ]:
spatial_mean_daily_precipitation_monthly = daily_precipitation['tp'].groupby('time.month')
data_list = []
data_99p_list = []
for m in range(1, 13):
    data_month = spatial_mean_daily_precipitation_monthly[m]
    data_month_99p = data_month.reduce(np.percentile, q=99, dim='time')
    data_list.append(data_month.values)
    data_99p_list.append(data_month_99p.values)
plt.figure(figsize=(10, 6))
sns.violinplot(data=data_list, cut=0, palette='Blues')
plt.plot(np.arange(0,12,1), data_99p_list, color='r', linestyle='--', label='99th Percentile')
plt.title('Multi-Annual Daily Precipitation Distribution per Month')
plt.xlabel('Month')
plt.ylabel('Precipitation [mm]')
plt.grid(axis='y')
plt.legend()
plt.show()

## Histogram of Daily Mean Precipitation with Percentiles

To understand the distribution of daily precipitation, we visualize a **histogram** of wet days (precipitation > 0.5mm).
This helps us analyze the **frequency of different precipitation amounts** and assess key statistical measures.


In [ ]:
# Optain array of precipitation data from the xarray
daily_mean = daily_precipitation['tp']
# filter out only wet days (daily precipitation > 0.5mm)
daily_mean_wet = daily_mean[(daily_mean > 0.5)]

# Calculate mean, median, and percentiles
mean_precip = daily_mean_wet.mean().values
median_precip = np.nanmedian(daily_mean_wet.values)
percentile_95 = np.nanpercentile(daily_mean_wet.values, 95)
percentile_99 = np.nanpercentile(daily_mean_wet.values, 99)

# Plot the histogram
plt.figure(figsize=(8, 6))
plt.hist(daily_mean_wet, bins=50, alpha=0.7, color='skyblue', edgecolor='black')

# Add lines for mean, median, and percentiles
plt.axvline(mean_precip, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean_precip:.2f}')
plt.axvline(median_precip, color='green', linestyle='dashed', linewidth=1, label=f'Median: {median_precip:.2f}')
plt.axvline(percentile_95, color='orange', linestyle='dashed', linewidth=1, label=f'95th Percentile: {percentile_95:.2f}')
plt.axvline(percentile_99, color='purple', linestyle='dashed', linewidth=1, label=f'99th Percentile: {percentile_99:.2f}')

# Plot legend, labels, and title
plt.legend()
plt.xlabel('Daily Mean Precipitation (mm)')
plt.ylabel('Frequency')
plt.title('Histogram of Daily Mean Precipitation with Percentiles')
plt.grid(True)
plt.show()

## Gamma Distribution Fit to Precipitation Data
We fit a **Gamma distribution** to the daily precipitation data and compare it to the empirical distribution.
The probability density function (PDF) of the Gamma distribution is given by:
$$ f(x; k, \theta) = \frac{x^{k-1} e^{-x/\theta}}{\theta^k \Gamma(k)} $$
where:
- $k$ is the shape parameter,
- $\theta$ is the scale parameter,
- $\Gamma(k)$ is the gamma function.

### Cumulative Distribution Function (CDF)

The **cumulative distribution function (CDF)**, which gives the probability that a random variable \( $X$ \) takes on a value less than or equal to \( x \), is given by:

$$ F(x; k, \theta) = \frac{1}{\Gamma(k)} \gamma(k, x / \theta) $$

where:
- \( $\gamma(k, x/\theta)$ \) is the **lower incomplete gamma function**.

The **CDF** is used for determining the probability of exceedance and calculating precipitation return levels.

In [ ]:
daily_mean = daily_precipitation['tp']
daily_mean_wet = daily_mean[(daily_mean > 0.5)]
shape, loc, scale = gamma.fit(daily_mean_wet)
x = np.linspace(0, daily_mean.max().values, 100)
pdf_fitted = gamma.pdf(x, shape, loc, scale)
plt.figure(figsize=(8, 6))
plt.hist(daily_mean_wet, bins=50, density=True, alpha=0.7, label='Observed Data')
plt.plot(x, pdf_fitted, 'r-', label='Fitted Gamma Distribution')
plt.xlabel('Precipitation (mm)')
plt.ylabel('Probability Density')
plt.title('Gamma Distribution Fit to Precipitation Data')
plt.legend()
plt.grid(True)
plt.show()

## Estimating Return Period Using the Fitted Gamma Distribution

To estimate the **return period** of extreme precipitation events, we model the **probability of exceedance** using the fitted **Gamma distribution**.


Not every day has precipitation, so we first determine the **probability of a wet day** (rainfall > 0.5mm):

$$ P_{\text{wet}} = \frac{n_{\text{wet}}}{n_{\text{all}}} $$

where $n_{\text{wet}}$ is the number of days with precipitation, $n_{\text{all}}$ is the total number of days in the dataset.


We select a **threshold precipitation level** (e.g., **50mm**) to analyze the frequency of extreme rainfall events.


The probability that **a precipitation event exceeds the threshold**, given that it is a wet day, is:

$$ P(x>X) = 1 - F(x) $$

where \( $F(x)$ \) is the **Cumulative Distribution Function (CDF)** of the fitted **Gamma distribution**.

Since precipitation only occurs on **wet days**, we adjust this probability by multiplying by \( $P_{\text{wet}}$ \):

$$ P(x>X)_{\text{wet}} = P(x>X) \cdot P_{\text{wet}} $$


The probability that at least one exceedance occurs in a **year (365 days)** is:

$$ P(x>X)_{\text{annual}} = 1 - (1 - P(x>X)_{\text{wet}})^{365} $$


The **return period** ($\tau$) represents the expected number of years between extreme precipitation events exceeding the threshold:

$$ \tau = \frac{1}{P_{\text{annual}}} $$


In [ ]:
# Calculate probability of rain days
n_all = daily_mean.size
n_wet = daily_mean_wet.size
p_wet = n_wet/n_all
# If we select a treshold we can model the return period based on the fitted distribution function
event_threshold = 50 #mm
# Optain the daily probability of threshold exeedance 
p_daily = 1 - gamma(shape, loc, scale).cdf(event_threshold)
# Transfrom this to an annual probability
p_annual = 1 - (1 - p_daily*p_wet)**365
# Claculate return period
return_period_model = 1/p_annual
# Print result
print(return_period_model)

## Extreme Value Analysis (EVA)

Extreme Value Analysis (EVA) is used to model rare events, such as extreme precipitation events, and estimate return levels over different time periods.

### Generalized Extreme Value (GEV) Distribution

The **Generalized Extreme Value (GEV)** distribution is commonly used in EVA and is defined as:

$$ F(x;\mu,\sigma,\xi) = \exp \left\{ - \left( 1 + \xi \frac{x - \mu}{\sigma} \right)^{-1/\xi} \right\}, \quad \text{for } 1 + \xi \frac{x - \mu}{\sigma} > 0 $$

where:
$ \mu $ is the location parameter (center of distribution),\
$ \sigma $ is the scale parameter (spread of distribution),\
$ \xi $ is the shape parameter, which determines the tail behavior:\
If $ \xi > 0 $, the distribution has a heavy tail (**Fréchet type**),\
If $ \xi = 0 $, it reduces to the **Gumbel distribution**,\
If $ \xi < 0 $, it has a finite upper bound (**Weibull type**).

### Applying EVA to Precipitation Data

We use the **pyextremes** library to:
- Identify **extreme precipitation events** (block maxima or peaks over threshold),
- Fit the **Generalized Extreme Value (GEV) model** to estimate return levels,
- Perform **diagnostic checks** to validate the model.

By applying this method, we can assess **climate risks** associated with extreme rainfall and understand long-term precipitation patterns.

In [ ]:
precip_values = daily_precipitation['tp'].sel(time=slice('1951-07-01', '2020-12-31')).to_pandas()
model = EVA(precip_values)
model.get_extremes(method='BM', block_size='365D')
model.plot_extremes()
model.fit_model()
summary = model.get_summary(return_period=[2, 5, 10, 25, 50, 100, 250, 500, 1000], alpha=0.95, n_samples=1000)
model.plot_diagnostic(alpha=0.95)
summary